In [30]:
import numpy as np

In [31]:
from loadData import load_data, TRAIN_DATA_POINTS, TEST_DATA_POINTS, N_CLASSES

train_data = load_data("data/ae.train", num_data_points=TRAIN_DATA_POINTS)
test_data = load_data("data/ae.test", num_data_points=TEST_DATA_POINTS)

Maximum number of rows of all datapoints: 29
Successfully created a NumPy array with shape: (270, 29, 12)
Maximum number of rows of all datapoints: 29
Successfully created a NumPy array with shape: (370, 29, 12)


In [32]:
train_time_steps, channels = train_data.shape[1:]
test_time_steps = test_data.shape[1]

In [33]:
from DynamicTimeWarping import DTW

distance = DTW(train_data[0], train_data[1])
distance

np.float64(693.629155)

In [ ]:
from processing import generate_class_matrix

k_folds = 9
class_matrix = generate_class_matrix(TRAIN_DATA_POINTS, N_CLASSES)

In [40]:
from DynamicTimeWarping import DTW

accuracies = []

for k_nn in range(1, 30):
    idxs = np.arange(TRAIN_DATA_POINTS)
    np.random.shuffle(idxs)

    train_data = train_data[idxs]
    class_matrix = class_matrix[idxs]

    data_folds = np.array(np.split(train_data, k_folds))
    class_folds = np.array(np.split(class_matrix, k_folds))

    misclassifications = 0
    for fold in range(k_folds):
        fold_misclassification = 0
        training_fold = np.concatenate([data_folds[idx] for idx in range(k_folds) if idx != fold])
        class_data = np.concatenate([class_folds[idx] for idx in range(k_folds) if idx != fold])

        for data_point, class_vect in zip(data_folds[fold], class_folds[fold]):
            distances = [DTW(data_point, training_fold[idx]) for idx in range(len(training_fold))]
            
            nearest_neighbors = np.zeros(N_CLASSES)
            for _ in range(k_nn):
                nearest_neighbor = np.argmin(distances)
                nearest_neighbors += class_data[nearest_neighbor]
                distances[nearest_neighbor] = np.inf

            prediction = np.argmax(nearest_neighbors)
            if prediction != np.argmax(class_vect):
                fold_misclassification += 1
                misclassifications += 1
        
        print(f"accuracy on fold {fold}:\t {100 - (fold_misclassification / data_folds.shape[1]) * 100}%")

    accuracy = 100 - (misclassifications / TRAIN_DATA_POINTS) * 100
    accuracies.append(accuracy)
    print(f"validation accuracy for k = {k_nn}:\t {accuracy}%")

accuracy on fold 0:	 6.666666666666671%
accuracy on fold 1:	 10.0%
accuracy on fold 2:	 13.333333333333329%
accuracy on fold 3:	 23.33333333333333%
accuracy on fold 4:	 3.3333333333333286%
accuracy on fold 5:	 23.33333333333333%
accuracy on fold 6:	 13.333333333333329%
accuracy on fold 7:	 20.0%
accuracy on fold 8:	 20.0%
validation accuracy for k = 1:	 14.81481481481481%
accuracy on fold 0:	 16.666666666666657%
accuracy on fold 1:	 6.666666666666671%
accuracy on fold 2:	 16.666666666666657%
accuracy on fold 3:	 20.0%
accuracy on fold 4:	 10.0%
accuracy on fold 5:	 16.666666666666657%
accuracy on fold 6:	 10.0%
accuracy on fold 7:	 16.666666666666657%
accuracy on fold 8:	 26.66666666666667%
validation accuracy for k = 2:	 15.555555555555557%
accuracy on fold 0:	 6.666666666666671%
accuracy on fold 1:	 20.0%
accuracy on fold 2:	 3.3333333333333286%
accuracy on fold 3:	 6.666666666666671%


KeyboardInterrupt: 

In [38]:
print(f"validation accuracy for k = {k_nn}:\t {100 - (misclassifications / TRAIN_DATA_POINTS) * 100}%")

validation accuracy for k = 3:	 90.0%
